In [133]:
# # -*- coding: utf-8 -*-
# from ckiptagger import data_utils # first use ckip, need to download data file
# data_utils.download_data_gdown("./")
from ckiptagger import WS, POS, NER, construct_dictionary

ws = WS("./ckip_data") # word segment
# pos = POS("./ckip_data") 
# ner = NER("./ckip_data") # entity recognition


import pandas as pd
import numpy as np
import time
import math
import re
import demoji
# demoji.download_codes()


#====== import jieba dict =====
import jieba
jieba.set_dictionary('dataset/dic_trad/dict.txt.big')
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt')


#====== selenium activate esc key ======
from selenium.webdriver import ActionChains # activate esc key
from selenium.webdriver.common.keys import Keys # activate esc key

#====== selenium cathc NoSuchElementException ======
from selenium.common.exceptions import NoSuchElementException   
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

#====== evaluation metrics ======
from sklearn.metrics import mean_squared_error #均方誤差
from sklearn.metrics import mean_absolute_error as mae #平方絕對誤差
from sklearn.metrics import r2_score

#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

Building prefix dict from /home/alvin/essay/dataset/dic_trad/dict.txt.big ...
Loading model from cache /tmp/jieba.ucbd952ca8a83da26cb61ea55c46a2e3b.cache
Loading model cost 0.745 seconds.
Prefix dict has been built successfully.


In [296]:
#====== get antusd senti dict ======
colNames = ['Word','Score','Pos','Neu','Neg','Non','Not']
antusd = pd.read_csv('dataset/ANTUSD_traditional/opinion_word.csv', names = colNames) 
word_senti_score = dict(zip(antusd['Word'],antusd['Score'])) # find score via word, 27221 words

# print(antusd)
# print(word_senti_score)

In [136]:
len(antusd)

28799

In [83]:
#====== emoji_sentiment_dict ======
emoji_sentiment = pd.read_csv('dataset/dic_trad/Emoji_Sentiment.csv') # read Emoji_Sentiment.csv
emoji_sentiment.drop(columns=['Unnamed: 0'], inplace = True)
emoji_sentiment_dict = dict(zip(emoji_sentiment['Emoji'],emoji_sentiment['Sentiment score'])) # key,value

In [4]:
def get_selenium_param(filename):
    paramList = []
    with open("dataset/ig_article/"+filename, "r") as f:
        for line in f:
            single_store_param = line.split(',')
            single_store_param[2] = single_store_param[2].replace('\n','')
            paramList.append(single_store_param)

    return paramList

In [262]:
#====== data preprocess - related function ======
def get_store_names(food_kind):
    store_names = [food_kind]
    # read ranked store names
    with open("dataset/ig_article/store_article/"+food_kind+"/ranked_store_names", "r") as f:    
        for line in f:
            store_names.append(line.strip())
        
    return store_names
    
def get_remove_word():
    word_list = []
    with open("dataset/dic_trad/removed_word.txt", "r") as f:    
        for line in f:
            word_list.append(line.strip())
        
    return word_list

In [298]:
#====== extend senti. dict ======
def self_extend_senti_dict(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score


#====== extend senti. dict ======
def self_extend_senti_dict_hotpot(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict_hotpot.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score    
        
#====== add emoji senti score ======
def add_emoji_senti_score(word_senti_score,emoji_sentiment_dict):
    word_senti_score.update(emoji_sentiment_dict)
#     return word_senti_score


self_extend_senti_dict(word_senti_score)
self_extend_senti_dict_hotpot(word_senti_score)
add_emoji_senti_score(word_senti_score, emoji_sentiment_dict)

In [88]:
emoji_sentiment_dict

{'😂': 0.221,
 '❤': 0.746,
 '♥': 0.657,
 '😍': 0.6779999999999999,
 '😭': -0.09300000000000001,
 '😘': 0.701,
 '😊': 0.644,
 '👌': 0.563,
 '💕': 0.632,
 '👏': 0.52,
 '😁': 0.449,
 '☺': 0.657,
 '♡': 0.669,
 '👍': 0.521,
 '😩': -0.368,
 '🙏': 0.41700000000000004,
 '✌': 0.46299999999999997,
 '😏': 0.332,
 '😉': 0.46299999999999997,
 '🙌': 0.5589999999999999,
 '🙈': 0.43200000000000005,
 '💪': 0.555,
 '😄': 0.42100000000000004,
 '😒': -0.374,
 '💃': 0.7340000000000001,
 '💖': 0.7120000000000001,
 '😃': 0.557,
 '😔': -0.146,
 '😱': 0.19,
 '🎉': 0.738,
 '😜': 0.455,
 '☯': 0.001,
 '🌸': 0.65,
 '💜': 0.654,
 '💙': 0.73,
 '✨': 0.35100000000000003,
 '😳': 0.018000000000000002,
 '💗': 0.657,
 '★': 0.28300000000000003,
 '█': -0.032,
 '☀': 0.465,
 '😡': -0.17300000000000001,
 '😎': 0.491,
 '😢': 0.006999999999999999,
 '💋': 0.691,
 '😋': 0.631,
 '🙊': 0.45899999999999996,
 '😴': -0.08,
 '🎶': 0.537,
 '💞': 0.7390000000000001,
 '😌': 0.48200000000000004,
 '🔥': 0.139,
 '💯': 0.12,
 '🔫': -0.19399999999999998,
 '💛': 0.7090000000000001,
 '💁': 0

In [84]:
print(len(word_senti_score),len(emoji_sentiment_dict))

27367 751


In [ ]:
#====== test ckip word segment ======

# text = '傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
# text = '"鬼金棒，我就跟！就這樣，我又跟我的前衣食父母吃飯！連吃兩天拉麵🍜一個胖的節奏#鬼金棒拉麵#辣正常麻增量#結果真的麻到我頭皮發麻 我到底要不要改機票呢😏"'
# text = '期盼那麼久才吃到，果然不負期望，無論是湯頭還是肉片都是最佳狀態，太享受了'
text = '好久不見！，體力一年不如一年，#20191101，服務態度超優的，海鮮😋,海底撈圈字遊戲送一盤肉,👍👍👍撈撈撈，💋💋💋,排隊排好久 超值得, 我要再來刷二訪 😋， 以愛吃重口味的我來說 麵條的硬度很讚 乾, 想吃很膩害的溏心蛋 這家沾麵真的很優,湯頭也很濃 我不愛沾麵 難波萬台灣美食成為我的愛店之一👍🏻好多人，不知道要排多久,麻的味道超級棒,推推推,湯頭還是很棒，豆芽菜的鑊氣沒了很難過，大概一陣子不會回訪了,good excelent,熱死第一名,可惜還是沒把湯喝完,應該很普通,不是我的最愛。'


ws_results = ws([text])
# pos_results = pos(ws_results)
# ner_results = ner(ws_results, pos_results)

print(ws_results)
# print(pos_results)
# for name in ner_results[0]:# print entity
#     print(name)

In [82]:
list(zip(emoji_sentiment['Emoji'],emoji_sentiment['Sentiment score']))

[('😂', 0.221),
 ('❤', 0.746),
 ('♥', 0.657),
 ('😍', 0.6779999999999999),
 ('😭', -0.09300000000000001),
 ('😘', 0.701),
 ('😊', 0.644),
 ('👌', 0.563),
 ('💕', 0.632),
 ('👏', 0.52),
 ('😁', 0.449),
 ('☺', 0.657),
 ('♡', 0.669),
 ('👍', 0.521),
 ('😩', -0.368),
 ('🙏', 0.41700000000000004),
 ('✌', 0.46299999999999997),
 ('😏', 0.332),
 ('😉', 0.46299999999999997),
 ('🙌', 0.5589999999999999),
 ('🙈', 0.43200000000000005),
 ('💪', 0.555),
 ('😄', 0.42100000000000004),
 ('😒', -0.374),
 ('💃', 0.7340000000000001),
 ('💖', 0.7120000000000001),
 ('😃', 0.557),
 ('😔', -0.146),
 ('😱', 0.19),
 ('🎉', 0.738),
 ('😜', 0.455),
 ('☯', 0.001),
 ('🌸', 0.65),
 ('💜', 0.654),
 ('💙', 0.73),
 ('✨', 0.35100000000000003),
 ('😳', 0.018000000000000002),
 ('💗', 0.657),
 ('★', 0.28300000000000003),
 ('█', -0.032),
 ('☀', 0.465),
 ('😡', -0.17300000000000001),
 ('😎', 0.491),
 ('😢', 0.006999999999999999),
 ('💋', 0.691),
 ('😋', 0.631),
 ('🙊', 0.45899999999999996),
 ('😴', -0.08),
 ('🎶', 0.537),
 ('💞', 0.7390000000000001),
 ('😌', 0.4820

In [6]:
# word_pos = dict(zip(ws_results[0],pos_results[0])) # combine word and pos tag

In [289]:
#====== data preprocess - implementation ======
def data_preprocess(articles): 
    # only get string comment & preprocess fans, likes
    all_comments, assigned_index, fans_count, likes  = [],[],[],[]  # assigned_index: denote the original dataframe index of comments
    for i in range(len(articles['text'])):     # remove '\n'
        if type(articles['text'][i]) == str:   # remove non-string
            
            # exclude noise article(ex:染髮)
            temp_word_list = jieba.lcut(articles['text'][i])
            remove_flag = False
            for removed_word in get_remove_word():
                if removed_word in temp_word_list:
                    remove_flag = True
                    break
            if remove_flag:
                continue
                    
            # if contain '\n', replace with ','
            if articles['text'][i].find('\n'): 
                all_comments.append(articles['text'][i].replace('\n','，'))
            else:
                all_comments.append(articles['text'][i])
            assigned_index.append(i)
            
#             # get assigned fans_count
#             if type(articles['fans_count'][i]) == np.float64 and math.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif type(articles['fans_count'][i]) == np.float64 and np.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif str(articles['fans_count'][i]).find('萬') != -1:
#                 wan_index = articles['fans_count'][i].find('萬')
#                 transform_count = (float(articles['fans_count'][i][:wan_index])) * 10000
#                 fans_count.append(transform_count)
#             else:
#                 fans_count.append(int(articles['fans_count'][i]))
            fans_count = []


            # get assigned likes
            temp_like = articles['likes'][i]
            if type(temp_like) == np.float64 or type(temp_like) == np.int64:
                likes.append(temp_like)
            elif type(temp_like) == str:
                if temp_like.isdigit():
                    likes.append(float(temp_like))
                else:
                    likes.append(1)
            else:
                likes.append(temp_like)
            
        # get article at most 120 post
        if len(all_comments) == 120:
            break
    
    return all_comments, assigned_index, fans_count, likes

In [8]:
#====== cal senti score by jieba ======
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

def get_senti_score(text):
    '''calculate one sentence's score'''
    ws_results = [jieba.lcut(text)] # word segment
    # get sentiword in dict
    total_score = 0
    for word in ws_results[0]:
        if word in word_senti_score:
            score = word_senti_score[word]
            total_score += score 
            print(word,score) # print each senti word's senti score
    
    
    print('sent_score:',total_score)
    if total_score >= 1:
        return 1
    elif total_score <= -1:
        return -1
    else:
        return total_score

In [ ]:
#====== cal senti score by ckip ======
# def get_senti_score(text):
#     ws_results = ws([text]) # word segment
#     total_score = 0
#     for word in ws_results[0]:
#         if word in word_senti_score:
#             score = word_senti_score[word]
#             total_score += score 
#             print(word,score) # print each senti word's senti score
    
    
#     print('sent_score:',total_score)
#     if total_score >= 1:
#         return 1
#     elif total_score <= -1:
#         return -1
#     else:
#         return total_score

In [9]:
#====== get each comment's score ======
def get_each_comment_score(all_comments):
    all_comments_scores = []
    for single_comment in all_comments:
        print('\ncomment:',single_comment)
        all_comments_scores.append(get_senti_score(single_comment))
        
    return all_comments_scores

In [10]:
#====== cal store score ======      # not yet consider no comment post's effect (maybe effective) 
def get_store_score(weight, each_comment_score, assigned_index):
    '''store score = comment score * likes(weight) /all likes'''
    weighted_score = 0
    total_weight = 0
    for i in range(len(each_comment_score)):
        if not np.isnan(weight[i]):
            weighted_score += (weight[i] * each_comment_score[i])
            total_weight += weight[i] 
            
    store_score = weighted_score / total_weight

    return store_score


In [11]:
def output_csv(filename, data_list, write_method):

    import csv

    # 開啟輸出的 CSV 檔案
    with open(filename, write_method, newline='') as csvfile:
        writer  = csv.writer(csvfile)   # 建立 CSV 檔寫入器
        for row in data_list: # format: two dimension
            writer.writerow(row)
            

In [ ]:
#====== remove foreign language post ======
for i,content in zip(articles[articles['text'].notnull()]['text'].index,
                     articles[articles['text'].notnull()]['text']):
    if not re.search(r'[\u4e00-\u9fff]+', content) and not demoji.findall(content): # check contain chinese
        print(i,content)
        # drop assigned row
        articles.drop(i, inplace=True) 
        
# reset dataframe index
articles.reset_index(drop=True) 

In [15]:
text = '這是一場五感都被放大的饗宴。﻿，﻿，有硬度及Q勁的麵體，🐠很喜歡，搭配辣少麻少的濃厚湯頭，非常過癮！鬼金棒的辣和麻是一流的，入口先感受到辣，隨之而來的麻會讓你感受到舌尖的震顫，建議第一次吃、不是很敢吃麻辣的朋友跟🐠一樣選擇辣少麻少。﻿，﻿，鬼金棒的拉麵，就是這樣強勁有力。﻿，﻿，叉燒肉也和大多數的拉麵店不同，選用大塊的五花肉，乍看之下有點像控肉（但絕對不是），肥瘦相間、軟嫩好入口，有別於一般薄片叉燒，鬼金棒就是這樣的霸氣！﻿，﻿，最後額外提一下，店裡的冰開水不知道為什麼，特別甘甜好喝啊🤣﻿，﻿，﻿，-·-·-·-·-·-﻿，🔺用餐請遵守店內顧客須知，大家一起當優良好客人！﻿，🔺吃完麵再加白飯100分！但麵的份量本身就蠻大了，要自己斟酌哦！﻿，——————————，﻿，🌟如果喜歡濃厚拉麵，或者吃膩了一般口味的拉麵，來鬼金棒走一遭吧☺️﻿，﻿，——————————，🏠辣麻味噌拉麵 鬼金棒﻿，📍104台北市中山區長安西路19巷2弄35號（近捷運中山站）﻿，🕐平日11:30-14:00/17:00-20:30，假日11:30-20:30﻿，，#辣麻味噌拉麵 #鬼金棒 #鬼金棒カラシビ味噌ラーメン #拉麵 #叉燒肉 #捷運美食 #捷運中山站 #ramen #ramennoodles 當然是想到鬼金棒💕，麻的味道超級棒，讓你一口接一口🤤，肉的份量也很足👍，湯頭非常濃郁‼️，吃#nightmarket #taiwan #taipei #noodles #stinkytofu 🤤#squid #seafood #dumplings #ningxia #nightlife,豪豪ㄘ 嘴唇超麻～～～,敲好ㄘ🌶🌶🌶🌶🍜'

# print(ws([text]))
demoji.findall(text)
for emoji in demoji.findall(text):
    if emoji_sentiment_dict.get(emoji):
        print(emoji,emoji_sentiment_dict[emoji])

🌟 0.327
🐠 0.414
🏠 0.5
👍 0.521
💕 0.632
🍜 0.395
📍 0.111


In [263]:
jieba.lcut('新娘美甲喝起來更香濃愛吃重口味的我來說嗚嗚嗚嗚可惜吃到後面有點膩今天沾麵日！白湯&海湯沾麵🍜')

['新娘',
 '美甲',
 '喝',
 '起來',
 '更',
 '香濃',
 '愛吃',
 '重',
 '口味',
 '的',
 '我',
 '來說',
 '嗚嗚',
 '嗚嗚',
 '可惜',
 '吃',
 '到',
 '後面',
 '有點',
 '膩',
 '今天',
 '沾麵',
 '日',
 '！',
 '白湯',
 '&',
 '海湯',
 '沾麵',
 '🍜']

In [288]:
articles

,web-scraper-order,web-scraper-start-url,article_link,article_link-href,text,user_info_page,user_info_page-href,post_count,fans_count,follow_others_count,likes
0,1587634668-4014,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/B1_hORJA0oO/,https://www.instagram.com/p/B1_hORJA0oO/,耶～終於有照片po了🥰\n我是臭宅 但是很喜歡跟你們出來～\n\n#閉關修煉中 #大家趕快擺...,jing___1995,https://www.instagram.com/jing___1995/,493 貼文,877,1045,51
1,1587634366-3999,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/B1rCy7knAVW/,https://www.instagram.com/p/B1rCy7knAVW/,螺絲瑪莉義麵坊\n台北市中山區南京西路12巷13弄9號\n11:30-14:30\n17:3...,yoga20170408,https://www.instagram.com/yoga20170408/,101 貼文,52,48,20
2,1587632134-3897,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/BzK1EdrHxXK/,https://www.instagram.com/p/BzK1EdrHxXK/,純色也是奢華驚人美😏\n❤️當月壽星，凡出示證件，現折200❤️\n👉🏻不得與其他優惠併用喔...,missbeauty_nail,https://www.instagram.com/missbeauty_nail/,778 貼文,1711,455,32
3,1587632566-3916,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/BzpZqwIjPjm/,https://www.instagram.com/p/BzpZqwIjPjm/,台灣 台北市 蘿絲瑪莉義麵坊\n凡點主餐餐後都會招待甜點\n推薦熔岩巧克力蛋糕😋好吃\n導航...,zoezoe9,https://www.instagram.com/zoezoe9/,577 貼文,390,NaN 追蹤中,NaN
4,1587633713-3969,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/B1JIJ00gWj-/,https://www.instagram.com/p/B1JIJ00gWj-/,餐廳的壁畫。\n我完全被電到了（迷倒）。,gentsuki,https://www.instagram.com/gentsuki/,1100 貼文,104,267,10
...,...,...,...,...,...,...,...,...,...,...,...
175,1587633411-3956,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/B0liWhNnaln/,https://www.instagram.com/p/B0liWhNnaln/,我眼中的妳和妳眼中的我😍❤️✨😘四個月後見！\n\n#bestie #friend4ever...,zengxisyuan,https://www.instagram.com/zengxisyuan/,2719 貼文,714,2039,38
176,1587633813-3974,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/B1LZ4iQAgza/,https://www.instagram.com/p/B1LZ4iQAgza/,短短ㄉ小聚會☺️☺️☺️\n下次集滿不知道何時欸\n#longtimenosee \n#fr...,lucy77827,https://www.instagram.com/lucy77827/,1069 貼文,325,554,40
177,1587634256-3994,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/B1lRqXDnsu4/,https://www.instagram.com/p/B1lRqXDnsu4/,🏠店名：螺絲瑪莉義麵坊RoseMary﻿\n﻿\n☎️電話：02-2521-9822﻿\n﻿...,88_food_trip,https://www.instagram.com/88_food_trip/,144 貼文,137,342,10
178,1587631872-3885,https://hackmd.io/dtfw3neQTRaAVUqCdgnaGQ,https://www.instagram.com/p/By7ExTtApjA/,https://www.instagram.com/p/By7ExTtApjA/,好朋友就是\n不常見面\n也不會因爲距離產生疏離感\n碰面總是有說不完的話題\n特別的緣份讓...,hsien_min,https://www.instagram.com/hsien_min/,1114 貼文,730,1320,NaN


In [305]:
#====== cal one store rating ======

#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score)
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


# get articles, followers
fileName = 'pasta/article_MINT Pasta 義大利麵' #
# fileName = 'pasta/article_螺絲瑪莉義麵坊' # 168
# fileName = 'pasta/article_村口微光' # 180


# fileName = 'hotpot/article_問鼎麻辣鍋忠孝店' # 147
# fileName = 'hotpot/article_滿堂紅bellavita店_松江店' # 101



# fileName = 'article_山嵐拉麵_公館店_台灣總店' # 85 post
# fileName = 'article_山嵐拉麵' # 33 post (remove)
# fileName = 'article_你回來啦 Okaeri 拉麵'# 96
# fileName = 'article_鬼金棒_中山北路' # 126
# fileName = 'article_麵屋壹の穴 ICHI' # 50 (remove)
# fileName = 'article_麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店' # 106
# fileName = 'article_柑橘 Shinn' # 69


articles = pd.read_csv(open('dataset/ig_article/store_article/'+fileName+'.csv')) 

# remove '\n', delete non-string
all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

# get each comment's score
all_comments_scores = get_each_comment_score(all_comments)

# get store rating
# print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
store_rating = get_store_score(likes, all_comments_scores, assigned_index)
print('\nstore final score:',store_rating) # weight: likes
print('post count:',len(all_comments_scores))

# output ramen rating 
# output_csv('ramen_rating.csv', [[fileName, store_rating]], 'a+')




comment: MINT PASTA 綠薄荷麵食坊🍝 ，高CP值❤海鮮麵量都多多👉推薦！，，每次來用餐都點白醬類義大利麵，這次來點比較不一樣的~❤粉紅醬起司雞肉麵~味道一樣很不錯！😍 不但麵都有入味，配料給的很有誠意價格也便宜，推薦給喜歡吃義大利麵的朋友們~CP值超棒的選擇💯💯💯 #必點阿薩姆奶茶，#超脆酥皮濃湯，#入口即化
薄荷 0.0
🍝 0.11699999999999999
❤ 0.746
👉 0.39
推薦 0.019754
不 -0.05
一樣 0.0540008
❤ 0.746
起司 0.0
味道 0.0
一樣 0.0540008
不錯 0.2
😍 0.6779999999999999
入味 0.0
誠意 0.5694408000000001
便宜 0.0
推薦 0.019754
喜歡 0.875
超棒 0.35
選擇 0.0210186
💯 0.12
💯 0.12
💯 0.12
入口即化 0.3
sent_score: 5.449968999999999

comment: ⭐️⭐️⭐️⭐️⭐️，⏭更多菜色，粉紅醬鮮蔬義大利麵🍝，檸檬紅茶🥤，蘑菇濃湯+蕃茄湯🥣，🚨，🏠mint pasta義大利麵，☎️02-29330811，🚈#景美站 ，#五辛素#蛋奶素#奶素#全素#方便素#素食#台北美食#義式
⭐ 0.58
⭐ 0.58
⭐ 0.58
⭐ 0.58
⭐ 0.58
🍝 0.11699999999999999
🚨 0.638
🏠 0.5
☎ 0.0
方便 0.1761116
素食 0.0
美食 0.16
sent_score: 4.491111599999999

comment: 評價如圖，FYI.
sent_score: 0

comment: 唔唔推薦平價美食😋，推薦上班族平日美食，畢竟不可能天天都大餐😅，菠菜起士雞肉筆尖麵 145元，海鮮焗飯 170元，人真的很多～，記得早點來唷！，地址：台北市文山區景興路274-2號，▶️再訪指數9/10，歡迎追蹤🙏，#taiwan #taipei #foodie#foodporn #foods #italian #noodles #travel #相機食先 ，#pictures #yummy #台北美食 #美食 #銅板美食
推薦 0.019754
美食 0.16
😋 0.25
推薦 0.019754
美

In [322]:
#====== MSE ======
# input store name: [a,b,c]
# output: [score.....]
def cal_all_store_rating(store_names):
    all_store_rating = []
    all_store_post_count = [] # store info dict.
    for i,name in enumerate(store_names[1:]):
        print('-----------------------------------')
        print(name)
        fileName = '/article_' + name
        articles = pd.read_csv(open('dataset/ig_article/store_article/'+store_names[0]+fileName+'.csv')) 

        # remove '\n', delete non-string
        all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

        # get each comment's score
        all_comments_scores = get_each_comment_score(all_comments)

        # get store rating
        # print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
        store_rating = get_store_score(likes, all_comments_scores, assigned_index)
        
        print('\nstore final score:',store_rating) # weight: likes
        all_store_rating.append(store_rating)
        all_store_post_count.append(len(all_comments))
        
    return all_store_rating,all_store_post_count



#===================================== main function =======================================
# ramen
# store_names = ['ramen','鬼金棒_中山北路','雞吉君拉麵','山嵐拉麵_公館店_台灣總店','你回來啦 Okaeri 拉麵',
#                '麵屋壹慶','勝王','五之神製作所','柑橘 Shinn','麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店']

# hotpot
# store_names = ['hotpot','辛殿麻辣鍋','海底撈火鍋','老四川巴蜀麻辣燙-南京店','這一鍋皇室秘藏鍋物',
#                '詹記麻辣火鍋 敦南店','新馬辣武昌店_新馬辣經典馬辣鍋','青花驕 麻辣鍋',
#                '馬辣頂級麻辣鴛鴦火鍋 漢口店','問鼎麻辣鍋忠孝店','滿堂紅bellavita店_松江店']

# select food kind to get store names
store_names = get_store_names('pasta')


#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score) 
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


# cal_all_store_rating
all_store_rating, all_store_post_count = cal_all_store_rating(store_names)


print('all store rating:',all_store_rating)
print('post count:',all_store_post_count)

-----------------------------------
JAI 宅

comment: 意外找到一間網美餐廳，來到這人人都是網美💛，網美聖地💓，每個角落都很好拍 餐點也好吃，玩到被店員提醒餐點涼掉就不好吃囉🥴，，#台中美食#taichung#jai宅#吃貨動起來#食在好好吃#非吃不可#美好食光# 潔妮 日常#台中景點#網美景點
意外 0.0
找到 0.0445837
網美 0.1
網美 0.1
💛 0.7090000000000001
網美 0.1
聖地 0.2
💓 0.664
好吃 0.37684395
提醒 0.013909999999999999
不 -0.05
好吃 0.37684395
美食 0.16
好吃 0.37684395
美好 0.8176861999999999
景點 0.0
網美 0.1
景點 0.0
sent_score: 4.08971175

comment: 第一次來是粉色頭髮，第二次來還是粉色😂，他們的青醬爆炸蛤蜊義大利麵真的推👈，#網美當起來😎
第一次 0.036000533
還是 0.1
😂 0.221
爆炸 0.0
👈 0.424
😎 0.1
sent_score: 0.881000533

comment: 190823 🥳
sent_score: 0

comment: 認真傾聽自己內在的聲音❤️
認真 0.5221302
傾聽 0.0
自己 0.0
❤ 0.746
sent_score: 1.2681301999999999

comment: -，0907，•橘醬爆炸蛤蜊義大利麵，•明太子炭烤雞肉串，•黑松露爆炸蛤蜊飯（加焗烤），•JAI現炸脆薯，-，義大利麵超級好吃，那個醬太猛了 吃完還會想再點一份，蛤蜊也真的爆炸多的，雞肉串是店員推薦的，很令人驚艷的雞肉串，咬下去 裡面超級超級嫩，又有滿滿的肉汁 加上外面的明太子 太好吃了，燉飯跟薯條 我就覺得普通了，來這裡點義大利麵就對了，-，而且他號稱台中最美的餐廳，裡面空間很大 各個地方都有精心佈置 絕對是會回訪的店❤️
爆炸 0.0
爆炸 0.0
超級 0.0676179
好吃 0.37684395
還會 0.14104389999999997
想 0.005
爆炸 0.0
推薦 0.019754
驚艷 0.2573937
下去 0.0
超級 0.067617

美食 0.16
✨ 0.35100000000000003
✨ 0.35100000000000003
✨ 0.35100000000000003
✨ 0.35100000000000003
及時 0.0815279
美妙 0.725
隨意 -0.1694408
美妙 0.725
sent_score: 2.9260871

comment: 網美店👣
👣 0.344
sent_score: 0.344

comment: 要吃檸檬沒有檸檬，要吃鮭魚沒有鮭魚，結果還是點青醬吃😋，感覺我的唯一選擇總是吃青醬，我很喜歡，但偶爾想吃別的口味，竟然沒有⋯，只能說生意太好嘍～～～，不過🍝還是很好吃👍🏻
沒有 -0.0921865
沒有 -0.0921865
還是 0.1
😋 0.25
感覺 0.02
唯一 0.0540008
選擇 0.0210186
總是 0.05
喜歡 0.875
想 0.005
竟然 -0.012536499999999999
沒有 -0.0921865
只能 0.0
太好 0.28767719999999997
不過 0.0
🍝 0.11699999999999999
還是 0.1
很好吃 0.4
👍 0.25
sent_score: 2.2406006

comment: 好像被森林圍繞著，一種放鬆舒服的用餐感覺！😬😋🥰#台中 #美食 #台湾 #台湾ひとり旅 #手機食先 #吃貨 #吃 #eat #yummy #taiwanfood #forest #photoshoot #foodie #igo #igfood #igdaily #ig_color #ig_great_pics #igfashion #instafood #instacool #instadaily #instaphoto #instastyle#ig_myshot#cake#甜品控#森林
好像 0.44268620000000003
放鬆 0.0
舒服 0.326906
感覺 0.02
😬 0.19399999999999998
😋 0.25
🥰 0.25
美食 0.16
yummy 0.1
sent_score: 1.7435922

comment: @yizhongstreet_food每日更新2次！逛一中之前別忘記有追蹤我喔❤❤，-，Google評分：4.3，個人評論：4.

類似 0.0
感覺 0.02
適合 0.775
聚餐 0.1
sent_score: 1.7186967000000002

comment: 帶我弟來吃晚餐 淋成落湯🐔
🐔 0.3
sent_score: 0.3

comment: Hun Pasta🍝，-，，平時工作繁忙，加上沒有公休日，只有一起排假進修的時候才能有機會跟同事一起坐下來好好吃個飯、聊個天。 -，，#橘醬爆炸蛤蜊義大利麵，#義式番茄花雕酒香爆炸蛤蜊，#明太子奶油炸蝦義大利麵，#混#hun#台中美食
🍝 0.11699999999999999
加上 0.16797105
沒有 -0.0921865
只有 0.0
才能 0.6160255
機會 0.3837311
好吃 0.37684395
爆炸 0.0
爆炸 0.0
美食 0.16
sent_score: 1.7293850999999998

comment: 很餓～～ #橘醬爆炸蛤蜊燉飯，#奶油海鮮燉飯🐙🦐🐟
爆炸 0.0
🐙 0.387
🐟 0.6890000000000001
sent_score: 1.076

comment: 20190707，好久沒有跟各位醜八怪們出門了，這麼久沒見你們一定很想我😘，還有更想我3個醜八怪的沒有來🤪，#hun混🍝
沒有 -0.0921865
醜八怪 -0.6166666670000001
沒 -0.1
一定 0.2731792
想 0.005
😘 0.701
還有 0.2078135
更想 0.07611160000000002
醜八怪 -0.6166666670000001
沒有 -0.0921865
🍝 0.11699999999999999
sent_score: -0.13760203400000004

comment: Dena染 ❤，，#pickme  #一中如果髮廊💈，#臺中 #一中 #推薦 #髮廊，#我是Dena #黛娜是我，#用心 #細心 #貼心，#Matrix #美國 #原裝進口，，Line ID : s840902，Facebook : 蔡宜臻，📞： 04-2223-1088
❤ 0.746
推薦 0.019754
用心 0.51575515
細心 0.565104
📞 0.4
sent_score: 2.2466131500000004

comment: -，大樹美女團終於聚

comment: 生日晚餐🎉，姐姐生日快樂♥ฅ'ω'ฅ
生日 0.0
🎉 0.738
生日快樂 0.383642475
♥ 0.657
sent_score: 1.778642475

comment: 景·聲 即將上檔，好吃😋，每個人都不一樣，每個人都有他美麗的地方，每個人就是該尊重每個人，【物外】趙南開 張婷雅 雙個展 「物外」意為跳脫事物本體之外的趣味，著重人對世界的觀察與理解。在兩位一輕一重、一簡一繁的詮釋之下，創作內容寄情於生活符號，透過節選、重組、微觀與編繪，運用玻璃與版畫層疊交錯的製作手法，營造自我的理想境地...，_，【物外】趙南開 張婷雅 雙個展，展覽日期︱2019.4.20 sat ─ 5.26 sun，開幕茶會︱4.21 sun 3:00pm，開放時間︱11:00am-7:00pm（星期一、二休館），展出地點︱二空間 台北市中山區松江路226巷6號，展覽介紹︱https://www.spacetwo.net，，#Taiwan #Art #Arts #Artwork #Artworks #Artlife #Artist #Artists #ArtFair #ArtistsOnInstagram #Artwork #ArtShow #Artstagram #ArtGallery #ContemporaryArt #Exhibition #Gallery #Like4Like #IloveArt #Museum #MordernArt #Artlovers #ArtofInstagram #ConceptArt #Artlife #InstaArt #Artdaily #Thailand #Japan #Korea
即將 0.0
好吃 0.37684395
😋 0.25
不 -0.05
一樣 0.0540008
美麗 0.6160439
尊重 0.0
跳脫 0.15500899999999998
趣味 0.48467250000000006
著重 0.3311378
觀察 0.163617
理解 0.6432104999999999
詮釋 -0.05
創作 0.049686900000000006
寄情 0.0
透過 0.18016529999999997
編繪 0.0
運用 0.21806455
版畫 0.0
營造 0.05857150000000

久 -0.1
好吃 0.37684395
選擇 0.0210186
天使 0.45
非常 0.0788721
適合 0.775
沒 -0.1
非常 0.0788721
喜愛 0.8875107
味道 0.0
可以 0.1551797
特製 0.05
味道 0.0
非常 0.0788721
相當 0.46013175
新鮮 0.28997490000000004
火候 0.0
控制 0.0
不會 -0.27884699999999996
很棒 0.0550611
👍 0.25
還有 0.2078135
想 0.005
時間 0.0550611
一定 0.2731792
美食 0.16
yummy 0.1
螞蟻 0.0
sent_score: 4.3295438

comment: -，✔️牛肝蕈菇燉飯$240，✔️時蔬奶油菠菜麵$240，✔️瑪格麗特披薩$159，✔️冰淇淋紅茶$55，✔️鹽埕冬瓜檸檬$50，#dor留，CP值：★★★★☆，📌台南市中西區友愛街115巷5號之7，#屎綾呷毋霸#屎綾呷台南 ，#台南美食#台南美食地圖#台南美食推薦#台南餐酒館，#台南聚餐
✔ 0.27
✔ 0.27
✔ 0.27
✔ 0.27
✔ 0.27
★ 0.28300000000000003
★ 0.28300000000000003
★ 0.28300000000000003
★ 0.28300000000000003
☆ 0.428
📌 -0.4
友愛 0.6497816999999999
美食 0.16
美食 0.16
美食 0.16
推薦 0.019754
聚餐 0.1
sent_score: 3.7595357000000003

comment: 想吃提拉米蘇好久了，終於吃到可以的❤️然後最大的收穫是聽到你的好消息🎊好替妳開心💋，，#餐點也可以#義大利麵#燉飯#dor留
想 0.005
終於吃到 0.3
可以 0.1551797
❤ 0.746
最大 0.0
收穫 0.0424385
好消息 0.3705737
🎊 0.721
開心 0.4341682
💋 0.691
可以 0.1551797
sent_score: 3.6205398

comment: 🏃‍♂️Running Men 🏃‍♂️ ❤️心靈休息站❤️ 一起~DORE留趣🤗，，#我知道我在錠嵂我知道我很重

實在 0.45718945
可能 0.0
不 -0.05
習慣 0.0830536
起司 0.0
最後 -0.0175984
可惜 -0.1875861
沒 -0.1
喜歡 0.875
起司 0.0
應該 0.016946400000000004
甜點 0.0
美食 0.16
美食 0.16
光復 0.24114760000000002
甜點 0.0
美食 0.16
熱門 0.3667333
美食 0.16
相機 0.0
yummy 0.1
sent_score: 3.0426847500000003

comment: 我們用了六年多的時間看一個孩子長大，大到可以跟我在UNIQLO選她用過喜歡的濾鏡拍照，還有下一個孩子長大，下下一個孩子長大，我們同時長大，孩子的媽在逛街，另一個阿姨在試穿衣服，我們交換一部份的日常，深入淺出的，相信各有各的選擇各有各的道理，相信他人眼中的適合自己的顏色，相信同一個星座做的事情很可能很像，也很可能完全不一樣，就是日常
時間 0.0550611
長大 0.0
可以 0.1551797
喜歡 0.875
還有 0.2078135
長大 0.0
長大 0.0
同時 0.23198600000000003
長大 0.0
逛街 0.0
衣服 0.0
相信 0.5148499
選擇 0.0210186
道理 0.6399749
相信 0.5148499
適合 0.775
自己 0.0
相信 0.5148499
可能 0.0
可能 0.0
完全 0.45943999999999996
不 -0.05
一樣 0.0540008
sent_score: 4.9690243

comment: 「玫瑰浪花瓦帕」﻿，本以為是一般舒芙蕾﻿，吃下才發現內餡很有層次阿～～～﻿，起司+麻吉 就是絕配🤤﻿，另外醬也是重點✨✨✨ ﻿，大馬士革玫瑰醬+荔枝果漿+覆盆子果泥 ﻿，酸酸甜甜 好喜歡❤️～ ﻿，﻿，等 @yi_jie.hsieh 二寶出來之後，我們 @fairyann777 再去二訪🥰 ﻿，﻿，﻿，﻿，﻿，﻿，🍰2018-06-22🍰
發現 0.11151989999999998
起司 0.0
絕配 0.18357895
🤤 0.15
重點 0.1391674
✨ 0.35100000000000003
✨ 0.35100000000000003
✨ 0.35

選擇 0.0210186
❤ 0.746
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
☎ 0.0
預約 0.0330536
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
⠀ 0.047
☕ 0.244
學會 0.5335119
師生 0.0
sent_score: 4.054868700000003

comment: ⭕️KAFFE CAFE⭕️，-，好食🤤🤤的義大利麵餐廳；一進門🚪就立刻被特殊設計的旋轉玻璃門所吸引目光，而白色與木頭色錯落的擺設再加上半開放式廚房的設計，更顯得空間舒暢寬廣，讓人心情放鬆。，-，青醬蛤蜊烤蕃茄🍅 ²³⁰ ⓃⓉ，@hsiang.22 所點的，蒜與九層塔交融的醬汁與軟彈適中的義大利麵條，還有粒粒渾厚飽滿咬下會爆漿的蛤蜊，雖嫌味道濃郁，但再搭配旁邊微酸的小番茄🍅恰到好處👍👍，-，奶油蘑菇雞肉脆餅🍪 ¹⁸⁰ ⓃⓉ，濃郁香醇的奶油香氣與軟彈的義大利麵條，還有酥脆可口（咬下去ㄎㄠˊㄎㄠˊㄎㄠˊ）的脆餅🍪，真是一絕。，-，平時找不到想吃什麼不妨可以來這邊吃吃看，據說已經營業了10年以上，肯定是具有一定水準和等級才能屹立不搖的吧！！，-，🏢 台南市東區大學路22巷3號，☎️06-2379876，⏲️11:00-21:00，❗️其實還有賣甜點和生菜沙拉，-，#台南 ，#台南美食 ，#台南義大利麵 ，#成大美食 ，#tainan ，#tainanfood ，#tainanitalianrestaurant pasta，#台南好吃
🤤 0.15
🤤 0.15
立刻 0.19531965
特殊 0.2399749
設計 -0.0134571
吸引 0.0571086
白色 0.0
加上 0.16797105
設計 -0.0134571
顯得 0.38963359999999997
舒暢 0.17

推薦 0.019754
合理 0.775
很多 0.28742399999999996
味道 0.0
甜點 0.0
驚艷 0.2573937
覺得 0.02
好吃 0.37684395
真心 0.6929350000000001
覺得 0.02
很好吃 0.4
好吃 0.37684395
美食 0.16
sent_score: 3.3861946

comment: 偶爾放縱，卻忘了拍到主菜...某人讓我分心😂
放縱 0.0
分心 0.0
😂 0.221
sent_score: 0.221

comment: -，🚇 #台北 #中山，🔖 #中山美食 #義大利麵，🔍 #10義式料理系列，💡 不負責評分🌕🌕🌕🌕🌑，👄 不負責評論，一定要預約的店，不管什麼時候人都很多，推粉紅醬義大利麵，微微辣好吃，甜點杏仁霜❤️，-，#Taiwanfoodie10 #Taipeifoodie10
美食 0.16
💡 0.6
不 -0.05
負責 0.0
🌕 0.556
🌕 0.556
🌕 0.556
🌕 0.556
🌑 0.45799999999999996
👄 0.474
不 -0.05
負責 0.0
一定 0.2731792
預約 0.0330536
不管 -0.5948276
很多 0.28742399999999996
微微 0.0657943
好吃 0.37684395
甜點 0.0
❤ 0.746
sent_score: 5.0034674500000005

comment: 頗好吃
好吃 0.37684395
sent_score: 0.37684395

comment: 本來要吃MOMO，結果又沒開，每次約壽喜燒就那麼剛好約不成😭😭，好吧螺絲瑪麗還是很好吃的，，#螺絲瑪麗 #布丁好吃
不成 -0.030393299999999998
😭 -0.09300000000000001
😭 -0.09300000000000001
還是 0.1
很好吃 0.4
好吃 0.37684395
sent_score: 0.66045065

comment: 好吃
好吃 0.37684395
sent_score: 0.37684395

comment: <研究生聚餐>，好久不見的研究夥伴，瞬間勾起好多回憶，可惜無法全員集合，希望沒來的下次要出現喔，哈哈哈
聚餐 0.1
研究 0


comment: 我的心有一天也想要可以奔放❤️
想要 0.27222515
可以 0.1551797
❤ 0.746
sent_score: 1.17340485

comment: 原本是選擇離停車較近的 #瑰覓 ，卻要候位半小時至一小時，留下電話說有位置會通知，結果完全沒打來（黑人問號，，而 #村口微光 本是抱持著既然要等那就去看看的心態，一開始沒打算吃這間，因為營業時間只到兩點，很幸運一去馬上就有空位，沒吃蛤蜊選了干貝，卻是新品仙人掌🌵中卷勝出，，吃飽喝足後，閨蜜完全無聲無息，，#taiwan #台灣 #台灣美食 ，#taichung #台中 #台中美食 ，#西屯區 #西屯美食 #村口微光 #逢甲義大利麵 ，#澎湖仙人掌中卷麵 #奶油明太子干貝義大利麵 ，#唐揚雞 #濃湯 #鮮奶茶 #焦糖奶酪 ，#taiwanese #taiwanesefood #girlstalk美食 #foodie ，#popyummy #foodstagram #igfood #howsayhow ，#啖酌是一種挑戰味蕾的方式
選擇 0.0210186
停車 0.0
小時 0.0
留下 0.27625879999999997
電話 0.0
通知 0.43632429999999994
完全 0.45943999999999996
沒 -0.1
抱持 0.016946400000000004
開始 0.1162332
沒 -0.1
打算 0.5035213
幸運 0.27024440000000005
馬上 0.11956545
空位 0.0
沒 -0.1
卻是 -0.1
新品 0.0
🌵 0.192
勝出 0.3191934
完全 0.45943999999999996
美食 0.16
美食 0.16
美食 0.16
美食 0.16
挑戰 0.0
sent_score: 3.43018585

comment: 【巧遇巷弄美食】🍽，#村口微光 #台中美食 ，#巷弄美食 #逢甲，#義大利麵 #燉飯
巧遇 0.0
美食 0.16
美食 0.16
美食 0.16
sent_score: 0.48

comment: 【台中 • 村口微光 】🦞，✔️金沙奶油中卷義大利麵  299元，✔️橄欖油浸蒜燜龍蝦奶油義大利麵  399元，📍主餐附紅茶，📍D套餐（濃湯、沙拉、鮮奶茶）99元，.，👉🏻二

東西 0.0
sent_score: 1.6042315500000002

comment: 這間好吃！菜單的創意程度令人咋舌，#花生湯圓青醬燉飯
好吃 0.37684395
創意 0.3620392
令人咋舌 0.0015428
sent_score: 0.7404259499999999

comment: 逢甲巷弄小店
sent_score: 0

comment: #eting台中 #eting逢甲
sent_score: 0

comment: 小巷內的義式料理，奶油味很香，煙燻鮭魚好鹹，還不錯吃，紅茶喝完杯子是愛心❤️，#尋找小鈞美食 ，謝謝你每次都來聽我抱怨
鹹 -0.1
不錯 0.2
愛心 0.7554457
❤ 0.746
美食 0.16
謝謝 0.2974824
抱怨 -0.425
sent_score: 1.6339280999999997

comment: ⋯收心⋯
sent_score: 0

comment: Lobster pasta #taichungfood
sent_score: 0

comment: 難得今天上早班，早下班，可以跟我的好姐妹去聚餐吃一下好料囉～～，我們沒有點最澎湃的龍蝦🦞餐點，這應該要帶老公來撥比較適合，哈哈～，所以我們倆點超正常口味的義式料理，但我覺得簡單好吃😋夠濃郁～還加了$99元的套餐，吃太飽了😊😊，孕婦差點走不動～～，我的好姐妹，竟然很認真幫我拍照，害我很害羞😳，很認真聊了近況，希望我們彼此越來越好&越來越棒唷😘😘，#台中美食 #逢甲美食 #義式料理 #孕媽咪生活 #pregnant ，Ps.這週很快速來到 #32weekspregnant 😱😱默默8個月，好驚人唷，感覺快倒數了😱
難得 0.0121305
可以 0.1551797
聚餐 0.1
沒 -0.1
澎湃 0.0
應該 0.016946400000000004
適合 0.775
哈哈 0.07218
正常 0.49881099999999995
覺得 0.02
簡單 0.026052
好吃 0.37684395
😋 0.25
😊 0.644
😊 0.644
差點 0.0
竟然 -0.012536499999999999
認真 0.5221302
害羞 -0.67738575
😳 0.018000000000000002
認真 0.5221

In [323]:
#====== get rank : pasta ======
print('====== Pasta rank ======')
store_rank = []
temp_rating = all_store_rating[:]
temp_rating.sort(reverse = True)
cnt = 1
for name,i in zip(store_names[1:],all_store_rating):
    rank = temp_rating.index(i)+1
    store_rank.append(rank)
    print('第'+str(cnt)+'名:',name,round(i,5),',',all_store_post_count[cnt-1],'post',',',rank)
    cnt += 1
    

    


print('\n新評分機制排名:',store_rank)
print('MSE:',MSE(np.array(range(0,10)),np.array(store_rank)))
print('r2_score:',r2_score(np.array(range(0,10)),np.array(store_rank)))
print('-------------------------------------')
# print('Google_rank: [9,2,5,4,3,7,1,8,6,10]')
# print('Google_MSE:',MSE(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))
# print('Google_r2_score:',r2_score(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))

====== Pasta rank ======
第1名: JAI 宅 0.86034 , 120 post , 1
第2名: MINT Pasta 義大利麵 0.19581 , 38 post , 10
第3名: HUN 混 0.76836 , 120 post , 5
第4名: La PASTA 義大利麵屋 0.42093 , 42 post , 9
第5名: MIGA KITCHEN PASTA_米家廚房 0.5723 , 87 post , 8
第6名: Dor留 手工義大利麵 - 友愛街店 0.80299 , 85 post , 3
第7名: petit doux．微兜 Café Bistro 0.77749 , 120 post , 4
第8名: KAFFE at HOME 0.74731 , 53 post , 6
第9名: 螺絲瑪莉義麵坊 0.70997 , 120 post , 7
第10名: 村口微光 0.81154 , 120 post , 2

新評分機制排名: [1, 10, 5, 9, 8, 3, 4, 6, 7, 2]
MSE: 20.2
r2_score: -1.4484848484848483
-------------------------------------


In [128]:
#====== get rank : hotpot ======
print('====== Hotpot rank ======')
store_rank = []
temp_rating = all_store_rating[:]
temp_rating.sort(reverse = True)
cnt = 1
for name,i in zip(store_names[1:],all_store_rating):
    rank = temp_rating.index(i)+1
    store_rank.append(rank)
    print('第'+str(cnt)+'名:',name,i,rank)
    cnt += 1
    

    
#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

print('\nstore_rank:',store_rank)
print('MSE:',MSE(np.array(range(0,10)),np.array(store_rank)))
print('r2_score:',r2_score(np.array(range(0,10)),np.array(store_rank)))
print('-------------------------------------')
print('Google_rank: [9,2,5,4,3,7,1,8,6,10]')
print('Google_MSE:',MSE(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))
print('Google_r2_score:',r2_score(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))

====== Hotpot rank ======
第1名: 辛殿麻辣鍋 0.7197687980639518 3
第2名: 海底撈火鍋 0.7112354059740985 4
第3名: 老四川巴蜀麻辣燙-南京店 0.6304563806839624 7
第4名: 這一鍋皇室秘藏鍋物 0.6733346068147888 5
第5名: 詹記麻辣火鍋 敦南店 0.5899759673788846 8
第6名: 新馬辣武昌店_新馬辣經典馬辣鍋 0.5138251770630505 9
第7名: 青花驕 麻辣鍋 0.8318692422505327 1
第8名: 馬辣頂級麻辣鴛鴦火鍋 漢口店 0.5115873664699272 10
第9名: 問鼎麻辣鍋忠孝店 0.6540240611774751 6
第10名: 滿堂紅bellavita店_松江店 0.7199912063061299 2

store_rank: [3, 4, 7, 5, 8, 9, 1, 10, 6, 2]
MSE: 16.6
r2_score: -1.0121212121212122
-------------------------------------
Google_rank: [9,2,5,4,3,7,1,8,6,10]
Google_MSE: 12.8
Google_r2_score: -0.5515151515151515


In [134]:
mae(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10]))

2.6

In [132]:
#====== get rank : ramen ======
print('====== Ramen ======')
store_rank = []
temp_rating = all_store_rating[:]
temp_rating.sort(reverse = True)
cnt = 1
for name,i in zip(store_names[1:],all_store_rating):
    rank = temp_rating.index(i)+1
    store_rank.append(rank)
    print('第'+str(cnt)+'名:',name,i,rank)
    cnt += 1
    

    
#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

print('\nstore_rank:',store_rank)
print('MSE:',MSE(np.array(range(1,10)),np.array(store_rank)))
print('r2_score:',r2_score(np.array(range(1,10)),np.array(store_rank)))
print('-------------------------------------')
print('Google_rank: [9,3,5,4,1,6,7,2,8]')
print('Google_MSE:',MSE(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))
print('Google_r2_score:',r2_score(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))

====== Ramen ======
第1名: 鬼金棒_中山北路 0.6962326955986844 7
第2名: 雞吉君拉麵 0.933045603531786 1
第3名: 山嵐拉麵_公館店_台灣總店 0.6722511674113475 8
第4名: 你回來啦 Okaeri 拉麵 0.8915706979777054 3
第5名: 麵屋壹慶 0.8193721374015748 4
第6名: 勝王 0.7906018630633683 6
第7名: 五之神製作所 0.8014299354043659 5
第8名: 柑橘 Shinn 0.8955572947335314 2
第9名: 麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店 0.6680849742424243 9

store_rank: [7, 1, 8, 3, 4, 6, 5, 2, 9]
MSE: 11.555555555555555
r2_score: -0.7333333333333334
-------------------------------------
Google_rank: [9,3,5,4,1,6,7,2,8]
Google_MSE: 13.555555555555555
Google_r2_score: -1.0333333333333332


In [115]:
# google ramen rating


print('Google Rating MSE:')
print('Google_MSE_ramen:',MSE(np.array(range(1,10)),np.array([9,3,5,4,1,6,7,2,8])))
print('Google_MSE_hotpot:',MSE(np.array(range(0,10)),np.array([9,2,5,4,3,7,1,8,6,10])))

Google Rating MSE:
Google_MSE_ramen: 13.555555555555555
Google_MSE_hotpot: 12.8


In [155]:
# store_names = ['辣麻味噌拉麵 鬼金棒','雞吉君拉麵','山嵐拉麵','你回來啦 Okaeri 拉麵']
# store_page_xpath = ['href="/explore/locations/541098880/"','href="/explore/tags/雞吉君拉麵/"','href="/explore/locations/610378299165578/"','href="/explore/locations/1009572868/okaeri/"']
# first_page_xpath = ['href="/p/B-Rf46-jXl_/"','href="/p/B-CEZjVl8GC/"','href="/p/B-Wy4mhnnvk/"', 'href="/p/B-KNu4oFwcv/"']
# get_page_count = [500,130,500,500]


# store_names = ['一蘭拉麵','麵屋壹の穴 ICHI','勝王']
# store_page_xpath = ['href="/explore/locations/255951018612300/"','href="/explore/locations/1719618628292660/ichi/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"','href="/p/B-ZXrQEnbZ-/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [1200,1000,1000]


# store_names = ['一蘭拉麵']
# store_page_xpath = ['href="/explore/locations/255951018612300/"']
# first_page_xpath = ['href="/p/B-ZW6yUASZL/"']
# get_page_count = [4000]


# store_names = ['柑橘 Shinn','五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/765135860311072/shinn/"','href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-ZpZrvnZru/"','href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [600,700,700,700]

# store_names = ['五之神製作所 台灣','麵屋壹慶','勝王']
# store_page_xpath = ['href="/explore/locations/1919809258303714/"','href="/explore/locations/308926743186267/"','href="/explore/locations/2062844753939787/"']
# first_page_xpath = ['href="/p/B-Y78zMBrmj/"','href="/p/B-ZM0FnJglR/"','href="/p/B-Zdt_rAfae/"']
# get_page_count = [700,700,700]

store_selenium_param = get_selenium_param('selenium_url')
store_names = []
store_page_xpath = []
first_page_xpath = []

for line in store_selenium_param:
    store_names.append(line[0])
    store_page_xpath.append(line[1])
    first_page_xpath.append(line[2])

    
print(first_page_xpath)
# store_names = [store_selenium_param[0][0]]
# store_page_xpath = [store_selenium_param[0][1]]
# first_page_xpath = [store_selenium_param[0][2]]
# get_page_count = [4000]
# print(first_page_xpath)

['href="/p/B_PgFUaDk2y/"', 'href="/p/B_RlyLqA0z8/"', 'href="/p/B_SHtKjJof5/"']


In [156]:
#===== scrape url from first post ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(3)

    # get url of current page
    url_list, post_date = [], []
    url_list.append([driver.current_url]) # get first page url
    time.sleep(1)
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
    time.sleep(3)

    # get 500 page url
#     flag = 0
#     for i in range(get_page_count[store_index]):
    for i in range(8000):
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click()
        else:
            break

        time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(5)
        post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
        stopDate = post_date[-1][0]
#         print(post_date[-1][0],type(post_date[-1][0]))
        
        # stop date = 2019.6.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '5':
            break
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list, 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], post_date, 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print(stopTime-startTime/60)

HUN 混finished!
MINT Pasta 義大利麵finished!
JAI 宅finished!
1561167483.6183445


In [40]:
#===== scrape url from assigned date ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(4)

    # get url of current page
    url_list, post_date, output_post_date = [], [], []
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]) # get first page date

    # get 500 page url
    for i in range(5000):
        
        stopDate = post_date[-1][0]
        
        if stopDate[:stopDate.find('年')] != '2019':
#             print(stopDate)
            
            if check_exists_by_xpath('//a[@tabindex="0"][2]'):
                driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
                time.sleep(3)
            else:
                break
            time.sleep(1)
            post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
            continue
            
            
#         time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(1)
        temp_date = [driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]
        post_date.append(temp_date)
        output_post_date.append(temp_date)
        
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
            time.sleep(4)
        else:
            break

        # stop date = 2019.6.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '5':
            break
        
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list[:-1], 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], output_post_date[:-1], 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print((stopTime-startTime)/60)

新馬辣武昌店finished!
新馬辣經典馬辣鍋finished!
青花驕麻辣鍋板橋縣民大道店finished!
馬辣頂級麻辣鴛鴦火鍋-中山店finished!
馬辣頂級麻辣鴛鴦火鍋 漢口店finished!
問鼎麻辣鍋忠孝店finished!
問鼎 麻辣鍋 養生鍋 粉絲俱樂部finished!
滿堂紅-松江店finished!
滿堂紅bellavita店finished!
153.04108297030132


In [259]:
# teststr = '2019年8月10日'
teststr = '3月23日'
if teststr[:teststr.find('年')] == '2019' and teststr[teststr.find('年')+1:teststr.find('月')] == '8':
    print('yes')

# combine csv

In [317]:
article1 = pd.read_csv('dataset/ig_article/store_article/pasta/article_MIGA KITCHEN PASTA.csv')
article2 = pd.read_csv('dataset/ig_article/store_article/pasta/article_米家廚房.csv')
combined_article = pd.concat([article1, article2], ignore_index=True)
combined_article.to_csv('dataset/ig_article/store_article/pasta/article_MIGA KITCHEN PASTA_米家廚房.csv')

# extend dictionary

In [413]:
# extend dictionary
word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "緯來體育台": 1,
    "鬼金棒": 3,
    "海底撈": 5
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0, '鬼金棒': 3.0, '海底撈': 5.0}), (5, {'緯來體育台': 1.0})]
